Group 13

Team Members:
21110083, 21175261,110005057

In [ ]:
!pip install transformers
!pip install datasets
!pip install sec-api
!pip install pdfplumber

     |████████████████████████████████| 4.0 MB 12.2 MB/s 
     |████████████████████████████████| 6.5 MB 44.7 MB/s 
     |████████████████████████████████| 596 kB 36.5 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 14.8 MB/s 
     |████████████████████████████████| 136 kB 67.0 MB/s 
     |████████████████████████████████| 212 kB 71.9 MB/s 
     |████████████████████████████████| 1.1 MB 38.8 MB/s 
     |████████████████████████████████| 127 kB 51.4 MB/s 
     |████████████████████████████████| 144 kB 52.9 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 58.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import pdfplumber
from sec_api import QueryApi
import requests
from datasets import Dataset, load_metric

from scipy import spatial
import numpy as np
import torch
import re
import os
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Note that notebook saves some files to your local Google drive at /content/drive/MyDrive/. If you would like it saved elsewhere please edit the PATH variable below.

In [ ]:
PATH = '/content/drive/MyDrive/'
%env PATH_ENV = $PATH

env: PATH_ENV=/content/drive/MyDrive/


First we download 10-K reports as pdf files by reading the Ticker column in the scores dataframe. Here we provide 6 examples we used in our Experiments section, and if you want to experiment with new data you can directly edit the scored DataFrame or upload a new csv file in the same format. ESG ratings were obtained from https://www.sustainalytics.com/esg-ratings. If adding a new company please make sure to check it has filed a 10-K report with the SEC in 2021 https://www.sec.gov/edgar/searchedgar/companysearch.html.

In [ ]:
KEY = "e667c4a0afe7c087fd112bfdd72e756220a543057a85fa5dd518743b5ef30708"  # you can use our key for pulling new documents. note if it has surpassed the request limit you will need to obtain a free api key from https://sec-api.io/
filing_file_type = 'pdf'

url = 'https://drive.google.com/file/d/1DX9mRgAhTuVcKiuUCQQmxch3PMc3qaVz/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
scores = pd.read_csv(path)
scores

,Company Name,Ticker,Sustainalytics Score,Sustainalytics Class,Industry
0,Credit Acceptance Corp.,CACC,24.3,Medium,Diversified Financials
1,WD-40 Co.,WDFC,31.1,High,Household Products
2,"10X Genomics, Inc.",TXG,22.6,Medium,Pharmaceuticals
3,Aemetis Inc,AMTX,47.6,Severe,Refiners & Pipelines
4,"2U, Inc.",TWOU,19.8,Low,Software & Services
5,Cimpress Plc,CMPR,8.5,Negligible,Commercial Services
6,"Nature's Sunshine Products, Inc.",NATR,36.1,High,Household Products
7,1-800-Flowers.com Inc,FLWS,22.4,Medium,Retailing


In [ ]:
queryApi = QueryApi(api_key=KEY)

# number of URLs of reports to retrieve
n = 8
tickers = scores["Ticker"][:n]

# Initialise dictionary to store retrieved URLs
data = {}
print('Downloading PDFs...')
# Loop through the n companies
for company in tickers:
    
    print(company)
    # Build up the query string
    ticker_string = f'ticker: {company}'
    full_string = ticker_string + ' AND filedAt:{2021-01-01 TO 2021-12-31} AND formType:\"10-K\"'
    query = {}
    query["query"] = full_string
    query_string = {}
    query_string["query_string"] = query
    full_query = {}
    full_query["query"] = query_string

    # Call the queryAPI to get company details
    filings = queryApi.get_filings(full_query)

    # Store company the URL in a dictionary
    data[company] = filings["filings"][0]["linkToFilingDetails"]

    # download and save report to sec-filings folder
    reader_url = "https://api.sec-api.io/filing-reader?"
    payload = {"token": KEY, "type": filing_file_type, "url": data[company]}
    r = requests.get(reader_url, params=payload)

    with open(PATH + company + '.' + filing_file_type , 'wb') as file:
      file.write(r.content)


CACC
WDFC
TXG
AMTX
TWOU
CMPR
NATR
FLWS


Next we use GloVe word embeddings downloaded from https://nlp.stanford.edu/projects/glove/ to expand our ESG vocabulary from our seed words. If you would like to test with other seed words, just change the 'esg_seed' variable directly or change the path to a csv file with the same format as ours below.

In [ ]:
url = 'https://drive.google.com/file/d/1b3LHriBpj_IkNlAj__ABWwqbBBjyTQLK/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
esg_seed = pd.read_csv(path)
esg_seed

,E,S,G
0,biodiversity,social,governance
1,carbon,diversity,management
2,environment,equality,structure
3,emission,inclusion,compensation
4,pollution,fair,executive
5,sustainable,fairness,employee
6,waste,inequality,relations
7,Co2,equity,equitable
8,deforestation,welfare,pay
9,greenhouse,rights,NaN


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ZfptQKqy_X7fgvRb10JYRYJ9luQ-ZVLa' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ZfptQKqy_X7fgvRb10JYRYJ9luQ-ZVLa" -O ${PATH_ENV}glove.txt && rm -rf /tmp/cookies.txt

--2022-04-06 23:00:45--  https://docs.google.com/uc?export=download&confirm=t&id=1ZfptQKqy_X7fgvRb10JYRYJ9luQ-ZVLa
Resolving docs.google.com (docs.google.com)... 108.177.126.101, 108.177.126.139, 108.177.126.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.126.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ht9kuu9injnhdhvjdob5a1r3k25fok9v/1649286000000/01788286570193106814/*/1ZfptQKqy_X7fgvRb10JYRYJ9luQ-ZVLa?e=download [following]
--2022-04-06 23:00:45--  https://doc-08-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ht9kuu9injnhdhvjdob5a1r3k25fok9v/1649286000000/01788286570193106814/*/1ZfptQKqy_X7fgvRb10JYRYJ9luQ-ZVLa?e=download
Resolving doc-08-5g-docs.googleusercontent.com (doc-08-5g-docs.googleusercontent.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to doc-08-5g-docs.googleusercontent.com (doc-

In [ ]:
def find_similar_words(emmbedes):

    nearest = sorted(emmbed_dict.keys(), key=lambda word: spatial.distance.euclidean(emmbed_dict[word], emmbedes))

    return nearest


In [ ]:
emmbed_dict = {}
with open(PATH + 'glove.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], 'float32')
        emmbed_dict[word] = vector

esg_new = []

for col in esg_seed.columns:

    word_list = []
    words = esg_seed[col].dropna()
    
    for word in words:
        try:
            word_list.append(find_similar_words(emmbed_dict[word.strip().lower()])[0:10])

        except AttributeError as e:
            print(e)

    # add back seed words
    word_list = np.concatenate((np.asarray(word_list).reshape(-1), np.asarray(esg_seed[col])))
    word_list = word_list[np.where(word_list.astype(str) != 'nan')]
    esg_new.append(np.unique(word_list))

In [ ]:
esg_new

[array(['Co2', 'achieve', 'achieving', 'aim', 'aims', 'biodiversity',
        'carbon', 'change', 'climate', 'co2', 'conservation', 'conserving',
        'create', 'creating', 'deforestation', 'deforestation ',
        'degradation', 'desertification', 'development', 'dioxide',
        'disposal', 'diversity', 'dump', 'ecological', 'ecology',
        'ecosystem', 'ecosystems', 'emission', 'emissions', 'emitted',
        'emphasizes', 'emphasizing', 'ensuring', 'environment',
        'environmental', 'environments', 'epa', 'erosion', 'fauna',
        'furthermore', 'garbage', 'gases', 'gasses', 'ghg', 'governance',
        'greenhouse', 'impact', 'importantly', 'landfills', 'logging',
        'monoxide', 'moreover', 'nature', 'nitrogen', 'overexploitation',
        'overgrazing', 'pollutant', 'pollutants', 'polluted', 'pollution',
        'promote', 'protection', 'recycling', 'reduce', 'reducing',
        'reforestation', 'sewage', 'siltation', 'smog', 'so2',
        'stewardship', 'sul

Upon experimentation, the full vocalary produced texts excerpts that were too long to process given our computational constraints. We therefore downselected a set of 14 most relevant words for the expanded vocabulary for retrieving text excerpts.

In [ ]:
print('Extracting relevant text from PDFs...')
for name in tickers:
    print(name)
    pdf = pdfplumber.open(PATH + name + '.pdf')
    all_text = ''

    for pdf_page in pdf.pages:
        single_page_text = pdf_page.extract_text()
        # separate each page's text with newline
        all_text = all_text + '\n' + single_page_text
        pdf.close

    keywords = ['co2','carbon dioxide','climate change','ESG','global warming','ecosystems','emission','pollut','diversity','fairness','exploita','governance','transparen','key person']  
    new_text=''
    for keyw in keywords:
        pos1 = re.findall(r'[A-Z][^.!?]*' + keyw + r'[^.!?]*',all_text,flags=re.DOTALL)
        new_text += '. '.join(pos1)
    newf = open(PATH + name + '.txt','w+')
    newf.write(new_text)
    newf.close
newf = open(PATH + name + '.txt','w+')
newf.write(new_text)
newf.close

Extracting relevant text from PDFs...
CACC
WDFC
TXG
AMTX
TWOU
CMPR
NATR
FLWS


<function TextIOWrapper.close>

Next we further clean the text and join it with the scores DataFrame. We deal with texts that are too long by exploding into multiple rows. 

In [ ]:
checkpoint =  'bert-base-cased'
num_labels = 5 # 5 classes

ESG scores and classes manually retrieved from Sustainalytics are stored separately to the text files retrieved using the SEC API. Here we load ESG scores and text files, clean the text files and join scores and text by Ticker to retrieve the full dataset.

In [ ]:
scores['Text'] = np.nan

def clean_text(text):
    text = text.replace('\n', ' ').replace('\t', '').replace(',', '').replace(';', '')
    #regex = '\xc2\xb7'
    text = re.sub('[^a-zA-Z0-9 \.]', ' ', text) # remove any characters other than letters, numbers, spaces, periods
    text = re.sub(' +', ' ', text) # remove repeated spaces
    text = text.strip()

    return text

#for f in os.listdir(TXT_PATH):
for f in tickers:
  try:
    row = scores.index[scores['Ticker'] == f][0]
    # read text files and remove any newline, tab, comma characters
    with open(PATH + f+'.txt') as txt_file:
      text = txt_file.read()

    text = clean_text(text)
    scores.loc[row, 'Text'] = text

  except IndexError as e:
    print('File ', f, ' not in scores csv')

scores = scores.dropna()
scores = scores.reset_index(drop=True)

In [ ]:
scores

,Company Name,Ticker,Sustainalytics Score,Sustainalytics Class,Industry,Text
0,Credit Acceptance Corp.,CACC,24.3,Medium,Diversified Financials,In addition governmental regulations which wou...
1,WD-40 Co.,WDFC,31.1,High,Household Products,In particular legislators consumers investors ...
2,"10X Genomics, Inc.",TXG,22.6,Medium,Pharmaceuticals,We view much of this total market opportunity ...
3,Aemetis Inc,AMTX,47.6,Severe,Refiners & Pipelines,New laws or regulations relating to the produc...
4,"2U, Inc.",TWOU,19.8,Low,Software & Services,We plan to scale these offerings by expanding ...
5,Cimpress Plc,CMPR,8.5,Negligible,Commercial Services,In terms of combating climate change we have e...
6,"Nature's Sunshine Products, Inc.",NATR,36.1,High,Household Products,
7,1-800-Flowers.com Inc,FLWS,22.4,Medium,Retailing,Table of Contents If the Company is unable to ...


BERT has a 512 token limit, so we split rows so that each row has a text of about 200 words. We join them back together later.

In [ ]:
def longtext_split(text):
  total = []

  if len(text.split()) // 150 > 0:
    n = len(text.split()) // 150
  else:
    n = 1

  for w in range(n):
    if w == 0:
      partial = text.split()[:200]
    else:
      partial = text.split()[w*150:w*150+200]
    total.append(' '.join(partial))

  return total

scores['text_split'] = scores['Text'].apply(longtext_split)
scores = scores.explode('text_split')
      

Load the BERT model for sequence classification \and tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)
model = BertForSequenceClassification.from_pretrained(checkpoint, problem_type="single_label_classification", num_labels=num_labels)

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217

Load our fine-tuned model

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-OMBMTVTEyrXzxF_qiW-UamoInNozBqd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-OMBMTVTEyrXzxF_qiW-UamoInNozBqd" -O ${PATH_ENV}weights.pt && rm -rf /tmp/cookies.txt

--2022-04-06 23:11:08--  https://docs.google.com/uc?export=download&confirm=t&id=1-OMBMTVTEyrXzxF_qiW-UamoInNozBqd
Resolving docs.google.com (docs.google.com)... 108.177.127.100, 108.177.127.113, 108.177.127.101, ...
Connecting to docs.google.com (docs.google.com)|108.177.127.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/2aq4utrhakvhnscifv4bkm95gfeppm34/1649286600000/01788286570193106814/*/1-OMBMTVTEyrXzxF_qiW-UamoInNozBqd?e=download [following]
--2022-04-06 23:11:09--  https://doc-14-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/2aq4utrhakvhnscifv4bkm95gfeppm34/1649286600000/01788286570193106814/*/1-OMBMTVTEyrXzxF_qiW-UamoInNozBqd?e=download
Resolving doc-14-5g-docs.googleusercontent.com (doc-14-5g-docs.googleusercontent.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to doc-14-5g-docs.googleusercontent.com (doc-

In [ ]:
model.load_state_dict(torch.load(PATH+'weights.pt', map_location=torch.device('cpu')))
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Transform string labels into integers and encode inputs

In [ ]:
def transform_labels(label):
  label = label['Sustainalytics Class']
  
  if label == 'Negligible':
    num = 0
  elif label == 'Low':
    num = 1
  elif label == 'Medium':
    num = 2
  elif label == 'High':
    num = 3
  elif label == 'Severe':
    num = 4

  return {'labels': num}


def encode(example):
    return tokenizer(example['text_split'], example['Industry'], truncation=True, padding='max_length') 

dataset = Dataset.from_pandas(scores)
dataset = dataset.map(encode, batched=True) # encode data
remove_columns = ['Sustainalytics Score', 'Company Name', 'Ticker', 'Sustainalytics Class', 'Text', '__index_level_0__', 'text_split', 'Industry']
dataset = dataset.map(transform_labels)# , remove_columns=remove_columns) # change target column name to labels

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ex/s]

Make predictions and compute metrics

In [ ]:
# arguments for Trainer
test_args = TrainingArguments(
    do_train = False,
    do_predict = True, 
    output_dir = PATH,
    dataloader_drop_last = False    
)

# init trainer
trainer = Trainer(
              model = model, 
              args = test_args)

outputs = trainer.predict(dataset)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, Text, Sustainalytics Score, Industry, Sustainalytics Class, Ticker, text_split, Company Name. If __index_level_0__, Text, Sustainalytics Score, Industry, Sustainalytics Class, Ticker, text_split, Company Name are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 24
  Batch size = 8


In [ ]:
metric = load_metric('accuracy')
def compute_metrics(eval_pred):
  logits, labels, _ = eval_pred
  predictions = np.argmax(logits, axis=-1)

  return metric.compute(predictions=predictions, references=labels)

print(compute_metrics(outputs))

metric = load_metric('f1')
def compute_metrics(eval_pred):
  logits, labels, _ = eval_pred
  predictions = np.argmax(logits, axis=-1)

  return metric.compute(predictions=predictions, references=labels, average=None)

print(compute_metrics(outputs))

{'accuracy': 0.625}
{'f1': array([0.        , 0.        , 0.63636364, 1.        , 0.83333333])}


In [ ]:
out = pd.DataFrame(np.argmax(outputs.predictions, axis=-1), columns=['Prediction Class'])
out['Sustainalytics Class'] = dataset['Sustainalytics Class']
out['Company Name'] = dataset['Company Name']
out['Ticker'] = dataset['Ticker']
out['Industry'] = dataset['Industry']
# take the mean of the predictions
out = out.groupby(['Ticker', 'Company Name', 'Industry', 'Sustainalytics Class']).mean().reset_index()

In [ ]:
temp = []
for num in out['Prediction Class']:
  num = round(num)
  if num == 0:
    label = 'Negligible'
  elif num == 1:
    label = 'Low'
  elif num == 2:
    label = 'Medium'
  elif num == 3:
    label = 'High'
  elif num ==4:
    label = 'Severe'

  temp.append(label)

out['Pred Class'] = temp
out = out.drop('Prediction Class', axis=1)
del temp

out

,Ticker,Company Name,Industry,Sustainalytics Class,Pred Class
0,AMTX,Aemetis Inc,Refiners & Pipelines,Severe,High
1,CACC,Credit Acceptance Corp.,Diversified Financials,Medium,Medium
2,CMPR,Cimpress Plc,Commercial Services,Negligible,Medium
3,FLWS,1-800-Flowers.com Inc,Retailing,Medium,Medium
4,NATR,"Nature's Sunshine Products, Inc.",Household Products,High,High
5,TWOU,"2U, Inc.",Software & Services,Low,Medium
6,TXG,"10X Genomics, Inc.",Pharmaceuticals,Medium,Medium
7,WDFC,WD-40 Co.,Household Products,High,High
